In [222]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import speech_recognition as sr
import pyttsx3
import openai
import requests
import json
from bs4 import BeautifulSoup as bs
from difflib import SequenceMatcher




In [223]:

openai.api_key_path = "key.txt"
cur_model = "text-davinci-002"
img = ["image", "picture", "photo", "landscape", "painting", "drawing", "art"]


# Define Brave path
brave_path = "C:/Program Files/BraveSoftware/Brave-Browser-Nightly/Application/brave.exe"
options = webdriver.ChromeOptions()
options.binary_location = brave_path

# set dl options
prefs = {"download.default_directory": "C:/Users/thewa/Desktop/projects/computational_neuroscience/AI_ML/projects/mun_ching/crawlers/content"}

options.add_experimental_option("prefs", prefs)

# Create new automated instance of Brave
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.set_window_size(100,100)

C:\Users\thewa\AppData\Local\Temp\ipykernel_31928\1986775532.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [157]:
def callGPT(prompt):

        completion = openai.Completion.create(
                    model=cur_model,
                    prompt=prompt,
                    temperature=0,
                    frequency_penalty=0.5,
                    max_tokens=20,


                )
        
        return completion.choices[0].text
    

In [225]:

def getGame(response):
    
    URL = "https://steamcommunity.com/id/yusufwadi/games?tab=all&xml=1"
    data = requests.get(url=URL)
    soup = bs(data.text, 'xml')
    names = soup.find_all("name")
    
    for name in names:
        if (SequenceMatcher(a=response,b=name.text).ratio()) > 0.75:
            engine.say("Now launching... " + name.text)
            engine.runAndWait()
            return name.parent.find("appID").text
    
    print("Game not found")

    #print("hello")
#getGame("noida")



    


In [227]:
    
# From here
def qCommand(x):

    x = x.lower()
    r2 = sr.Recognizer()
   
    if 'youtube' in x:
    
           
            driver.get('https://www.youtube.com/')

            with sr.Microphone() as source:
                engine.say("What do you want to see?")
                engine.runAndWait()
                print('')
                audio = r2.listen(source)
                keyword = r2.recognize_google(audio)
                print(keyword)
      

            YOUR_PROMPT = "Extract a search query from the following prompt: " + '"' + keyword + '"'
 
            response = callGPT(YOUR_PROMPT)



            elem = driver.find_element('xpath', '//input[@id="search"]')
            elem.click()
            elem.send_keys(response, Keys.RETURN)

                

            #tryAudio(audio)

            # Till here is the code to run a YouTube vid
    elif 'google' in x:
           
            driver.get('https://www.google.com/')

            with sr.Microphone() as source:
                engine.say("What do you want to search for?")
                engine.runAndWait()
                print('')
                audio = r2.listen(source,timeout=5)
                keyword = r2.recognize_google(audio)
                print(keyword)
            YOUR_PROMPT = "Extract a search query from the following prompt: " + '"' + keyword + '"'

            completion = openai.Completion.create(
                model=cur_model,
                prompt=YOUR_PROMPT,
                temperature=0,
                frequency_penalty=0.5,
                max_tokens=20,


            )

            response = callGPT(YOUR_PROMPT)



            elem = driver.find_element('xpath', '//input[@class="gLFyf gsfi"]')
            elem.click()
            elem.send_keys(response, Keys.RETURN)

            #tryAudio(audio)
            
            
            # if img in x:

                
    elif 'play' in x:

        print(x)
        
        prompt = "Extract the name of the game from the prompt: " + x

        response = callGPT(prompt)

        gameID = getGame(response)

        driver.get("steam://rungameid/" + gameID)
        
        driver.switch_to.alert.accept
    

        #tryAudio(audio)

    else:
            print("command not recognized")


    



In [229]:

r3 = sr.Recognizer()

engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[0].id)
noCom = True

print('Listening...')
engine.say("Hello. I am Idaw. What can I do for you today?")
engine.runAndWait()

with sr.Microphone() as source:
        r3.adjust_for_ambient_noise(source,duration=0.5)
        audio = r3.listen(source,phrase_time_limit=2)
        phrase = r3.recognize_google(audio)

qCommand(phrase)



Listening...


UnknownValueError: 

# Voice Assist